In [3]:
#df = pd.read_csv(r'H:\MSC Data Science\Dissertation\data\PPG_FieldStudy\S1\S1_activity.csv')
#d = dtale.show(df)
#d
#共15个测试者，S5已知异常（可验证），取出一个测试者作为测试集，其他13个测试者的数据混合到一起，划分训练集和验证集（参考分心驾驶员检测）
#在胸部传感器信号数据中， ‘EDA’, ‘EMG’ and ‘Temp’无效，仅 ACC, ECG, RESP可用
#两种预测方式，1是把8秒平均心率视为每一秒都一样，去预测每秒，2是直接合并8秒的信号，预测8秒的平均心率，有准确的八秒平均也有每秒的
#异常检测
#先监测相关性，然后用滑窗来扩展变量数量
import numpy as np
import pandas as pd
import pickle



In [5]:
def load_data(file_path):
    S = pickle.load(open(file_path, 'rb'), encoding='latin1')#load pkl file as origin data
    
    signal = pd.DataFrame(S["signal"])
    chest = signal['chest']
    wrist = signal['wrist']

    label = pd.DataFrame(S['label'])
    base_len = len(label)
    label.columns = ['label']
    label.reset_index(drop = True, inplace=True)

    chest_ACC = pd.DataFrame(chest['ACC'])
    chest_ACC = chest_ACC.iloc[::1400, :]
    chest_ACC.columns = ["chest_ACC_x", "chest_ACC_y", "chest_ACC_z"]
    chest_ACC = chest_ACC[0:base_len]
    chest_ACC.reset_index(drop = True, inplace=True)
    
    chest_Resp = pd.DataFrame(chest['Resp'])
    chest_Resp = chest_Resp.iloc[::1400, :]
    chest_Resp.columns = ["chest_Resp"]
    chest_Resp = chest_Resp[0:base_len]
    chest_Resp.reset_index(drop = True, inplace=True)
    
    chest_ECG = pd.DataFrame(chest['ECG'])
    chest_ECG = chest_ECG.iloc[::1400, :]
    chest_ECG.columns = ["chest_ECG"]
    chest_ECG = chest_ECG[0:base_len]
    chest_ECG.reset_index(drop = True, inplace=True)
    
    data_chest = pd.concat([chest_ACC], sort=False)
    data_chest["Resp"] = chest_Resp
    data_chest["ECG"] = chest_ECG
    data_chest.reset_index(drop=True, inplace=True)

    wrist_ACC = pd.DataFrame(wrist['ACC'])
    wrist_ACC = wrist_ACC.iloc[::64, :]
    wrist_ACC.columns = ["wrist_ACC_x", "wrist_ACC_y", "wrist_ACC_z"]
    wrist_ACC = wrist_ACC[0:base_len]
    wrist_ACC.reset_index(drop = True, inplace=True)
    
    wrist_EDA = pd.DataFrame(wrist['EDA'])
    wrist_EDA = wrist_EDA[::8]
    wrist_EDA = wrist_EDA[0:base_len]
    wrist_EDA.columns = ["wrist_EDA"]
    wrist_EDA.reset_index(drop = True, inplace=True)

    wrist_BVP = pd.DataFrame(wrist['BVP'])
    wrist_BVP = wrist_BVP.iloc[::128, :]
    wrist_BVP = wrist_BVP[0:base_len]
    wrist_BVP.columns = ["wrist_BVP"]
    wrist_BVP.reset_index(drop = True, inplace=True)
    
    wrist_TEMP = pd.DataFrame(wrist['TEMP'])
    wrist_TEMP = wrist_TEMP[::8]
    wrist_TEMP = wrist_TEMP[0:base_len]
    wrist_TEMP.columns = ["wrist_TEMP"]
    wrist_TEMP.reset_index(drop = True, inplace=True)
    
    data_wrist = pd.concat([wrist_ACC], sort=False)
    data_wrist["wrist_BVP"] = wrist_BVP
    data_wrist["wrist_TEMP"] = wrist_TEMP
    data_wrist.reset_index(drop = True, inplace=True)
    
    
    data_signal = data_chest.join(data_wrist)
    for k,v in S["questionnaire"].items() :
        data_signal[k] = v
    
    rpeaks = S['rpeaks']
    counted_rpeaks = []
    index = 0 # index of rpeak element
    time = 175 # time portion
    count = 0 # number of rpeaks

    while(index < len(rpeaks)):
        rpeak = rpeaks[index]

        if(rpeak > time): # Rpeak appears after the time portion
            counted_rpeaks.append(count)
            count = 0
            time += 175

        else:
            count += 1
            index += 1
    # The rpeaks will probably end before the time portion so we need to fill the last portions with 0
    if(len(counted_rpeaks) < np.size(data_signal, axis = 0)):
        while(len(counted_rpeaks) < np.size(data_signal, axis = 0)):
            counted_rpeaks.append(0)
    peaks = pd.DataFrame(counted_rpeaks)
    peaks.columns = ["Rpeaks"]
    data_signal = data_signal.join(peaks)
    
    activity = pd.DataFrame(S["activity"]).astype(int)
    activity.columns = ["Activity"]
    data_signal = data_signal.join(activity)
    

    #label = pd.DataFrame(np.repeat(label.values,8,axis=0))
    #label.columns = ["Label"]
    #if(np.size(label, axis = 0) < np.size(activity, axis = 0)):
    #    mean = label.mean()
    #    while(np.size(label, axis = 0) < np.size(activity, axis = 0)):
    #        label = label.append(mean, ignore_index=True)
    
    data_signal = data_signal.join(label)
    
    data_signal['Subject'] = S["subject"]
    return data_signal

In [8]:

dataframes = {}
for i in range(1,16):
    dataframes['S' + str(i)] = load_data('PPG_FieldStudy/S' + str(i) + '/S' + str(i) + '.pkl')
dataframes.keys() 

dict_keys(['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15'])

In [12]:
S1 = load_data('./PPG_FieldStudy/S1/S1.pkl')
data = dataframes['S1']
for i in range(2,16):
    data = data.append(dataframes['S' + str(i)])
data.shape

(64697, 20)

In [13]:
data

,chest_ACC_x,chest_ACC_y,chest_ACC_z,Resp,ECG,wrist_ACC_x,wrist_ACC_y,wrist_ACC_z,wrist_BVP,wrist_TEMP,WEIGHT,Gender,AGE,HEIGHT,SKIN,SPORT,Rpeaks,Activity,label,Subject
0,0.8564,-0.0678,-0.3656,4.441833,0.015610,-0.765625,-0.078125,0.671875,7.28,32.13,78.0,m,34,182.0,3,6,0,0,49.611369,S1
1,0.8520,-0.0666,-0.3826,2.915955,0.159714,-0.750000,-0.078125,0.671875,6.34,32.15,78.0,m,34,182.0,3,6,1,0,50.323992,S1
2,0.8300,-0.0102,-0.3502,-1.623535,0.014603,-1.046875,-0.046875,0.843750,7.70,32.15,78.0,m,34,182.0,3,6,0,0,52.708336,S1
3,0.7630,0.0552,-0.6070,0.416565,-0.307617,-0.859375,-0.390625,0.390625,-120.35,32.15,78.0,m,34,182.0,3,6,0,0,55.640794,S1
4,0.8894,-0.0520,-0.2520,0.198364,0.604111,-1.015625,-0.078125,0.140625,-7.06,32.15,78.0,m,34,182.0,3,6,0,0,57.658406,S1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3961,0.7874,-0.0146,-0.5400,1.446533,0.486740,-0.984375,0.156250,0.234375,7.30,34.03,79.0,m,28,183.0,2,5,1,2,75.475622,S15
3962,0.8936,-0.1080,0.4138,10.545349,-0.124466,-0.562500,-0.031250,0.718750,117.29,34.00,79.0,m,28,183.0,2,5,0,2,77.524511,S15
3963,0.9012,0.0262,-0.1576,-7.260132,-0.138702,-0.375000,-0.203125,0.875000,39.55,34.00,79.0,m,28,183.0,2,5,1,2,78.714945,S15
3964,0.8966,0.0104,-0.2252,1.187134,-0.288849,-1.421875,-0.046875,0.171875,-21.28,34.00,79.0,m,28,183.0,2,5,0,2,80.413954,S15


In [11]:
data.to_csv("./data/dataset_sliced.csv", index=False)